In [1]:
import requests
from bs4 import BeautifulSoup
import pymysql
import re

In [2]:
def get_job_info(trabajo_link):
    """El input es un enlace que dirige a una única oferta de trabajo. Devuelve un tuple con la información más relevante: descripción del anuncio y del trabajo"""
    anuncio_list = []
    description_list = []
    import requests
    from bs4 import BeautifulSoup
    try:
        response = get_page(trabajo_link)
        result_page = BeautifulSoup(response.content,'lxml')
        anuncio_list = get_anuncio_list(result_page)
        description_list = get_job_description(result_page)
        empresa_name = get_name_empresa(result_page)
        output = unify_lists([empresa_name, anuncio_list, description_list])
        return output
    except:
        return None

In [3]:
def get_page(url):
    response = requests.get(url)
    if not response.status_code == 200:
        print("It didn't work")
    return response

In [4]:
def get_anuncio_list(result_page):
    '''This function recover the information concerning ID Empleo, Categoria, Ubicación, etc'''
    anuncio_list = []
    for info in result_page.find_all('div', class_='smallListingInfo'):
        if not 'Primera' in info.get_text(): #Exclude 'Primera Publicacion'
            if not 'Contrato' in info.get_text():
                anuncio_item = info.get_text()
                anuncio_item = anuncio_item[anuncio_item.find(':')+2:]
                anuncio_list.append(anuncio_item.replace('\n',''))  #Get items without \n
                #anuncio_list.append(info.get_text().replace('\n',''))
    return anuncio_list

In [5]:
def get_job_description(result_page):
    text = ''
    for paragraph in result_page.find_all('p'):
        if paragraph.text != "\n":
            text = text + paragraph.text
    text = clean_text(text)
    return text

In [6]:
def get_name_empresa(result_page):
    Enterprise_information = result_page.find('div', class_ = 'compProfileInfo')
    return Enterprise_information.find('strong').get_text()

In [7]:
def clean_text(text):
    import unidecode
    text = text.lower().replace('\n','') #Get items without \n
    text = re.sub("(\(\w+\))", '', text) #Delete parenthesis
    text = unidecode.unidecode(text) #Delete accents
    text = re.sub('[^A-Za-z0-9\s]','',text) #Delete anything but letters, numbers and spaces
    return text

In [8]:
def unify_lists(job_list):
    new_list = []
    new_list.append(job_list[0])
    for element in job_list[1]:
        new_list.append(element)
    new_list.append(job_list[2])
    return new_list

In [9]:
trabajo_link = "https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/695472/Analista-de-Tesorería.html?searchId=1545862649.53&page=1"
response = get_page(trabajo_link)
result_page = BeautifulSoup(response.content,'lxml')
anuncio_list = get_anuncio_list(result_page)
description_list = get_job_description(result_page)
empresa_name = get_name_empresa(result_page)
output = unify_lists([empresa_name, anuncio_list, description_list])

In [10]:
output

['NOSSAMODA SRL',
 '695472',
 'Santa Cruz de la Sierra, Bolivia',
 'Contabilidad y Economía ',
 'De Bs. 3.000 a Bs. 3.999 ',
 '28 Enero 2019',
 '27 Febrero 2019',
 'analista de tesoreria formacion titulado en auditoria financiera contaduria publica contaduria general o ramas afines experiencia y conocimientos  tres  anos de experiencia laboral en cargos de igual responsabilidad  elaboracion de conciliaciones bancarias y arqueos de caja elaboracion de flujos de caja gestion de pagos a proveedores locales y extranjeros seguimiento de cuentas pendientes de cobro y depositos de cobranzas del dia  preparacion de estados de cuentas por cobrar y pagar y control de emision de cheques registro contable de pagos a proveedores sueldos finiquitos etc  registro contable de cobros a clientes depositos transferencias etc  conocimiento en sistema contable sai   solidos manejo de windows office excel powerpoint outlook y word competencias  responsabilidad  organizacion  atencion a los detalles las pers

In [11]:
def trabajopolis(job_description):
    """ A partir de una (o más) palabras claves  """
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    for job in job_description:
        #url = "http://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D="+ job + "&page=1&listings_per_page=1000"
        url = "https://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D=cajero&page=1&listings_per_page=10"

        response = requests.get(url)
        if response.status_code == 200:
            print("The request response cycle was successful")
            results_page = BeautifulSoup(response.content,'lxml')
            #trabajo_list = list()
            trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
            for anuncio in trabajos:
                trabajo_link = anuncio.find('a').get('href')
                trabajo_name = anuncio.find('b').get_text()
                print(trabajo_link)
             #   trabajo_list.append((trabajo_name,trabajo_link))
            #for trabajo in trabajo_list:
                output = get_job_info(trabajo_link)
                trabajos_list.append((output,trabajo_name,trabajo_link))
            #I need to create a list that save all the extracted data
        else:
            print("The request response cycle work didn't work")
        time.sleep(10) 
    return trabajos_list

In [12]:
def trabajopolis2():
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    url = "https://www.trabajopolis.bo/search-results-jobs/?searchId=1546400657&action=search&page=1&listings_per_page=1500"
    response = requests.get(url)
    if response.status_code == 200:
        print("The request response cycle was successful")
        results_page = BeautifulSoup(response.content,'lxml')
        trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
        for anuncio in trabajos:
            trabajo_link = anuncio.find('a').get('href')
            trabajo_name = anuncio.find('b').get_text()
            print(trabajo_name)
            output = get_job_info(trabajo_link)
            trabajos_list.append((output,trabajo_name,trabajo_link))
            #I need to create a list that save all the extracted data
    else:
        print("The request response cycle work didn't work")
    return trabajos_list

In [13]:
Results = trabajopolis2()

The request response cycle was successful
Vendedor(a)
Sub Gerente de Recursos Humanos - Santa Cruz
Encargado(a) de Venta de Sala
Encargado(a) Almacén
Analista en Comunicación Estratégica
Jefe de Producción
Auxiliar Contable
Ayudante de Almacén
Diseñador(a) Gráfico(a) - Santa Cruz
Sub Gerente Nacional Técnico y de Reaseguro
Conductores de Maquinaria Pesada
Practicantes Industriales - Oruro
Supervisor(a) de Seguridad
Auxiliar de Crédito - Cochabamba
Vendedor(a)
Pasantía en el Departamento Técnico - La Paz
Promotores(as) de Ventas - Oruro
Promotores(as) de Ventas - Potosí
Promotores(as) de Ventas - La Paz
Promotores(as) de Ventas - Cochabamba
Promotores(as) de Ventas - Santa Cruz
Encargados(as) de Auditoría
Oficial de Crédito Júnior - Agencia San Miguel
Jefe Customer Service & Logistics
Supervisor(a) de Almacén - Santa Cruz
Supervisor(a) de Almacén
Ejecutivo(a) de Cuenta Banca Personas - Cochabamba
Key Accounts Manager Assistant - La Paz
Ejecutivo(a) de Ventas - Canal Moderno (Supermercad

Supervisor(a) Operativo(a) Administrativo(a) - Potosí
Jefe de Contabilidad
Analista de Riesgo Crediticio Banca PyME
Project Manager - Santa Cruz
Asistente de Jefatura Comercial
Pasante para el Área de Recursos Humanos - Cochabamba
Diseñador(a) Gráfico(a)
Conductor(a)
Asesor(a) Comercial Freelance - Santa Cruz
Auxiliar Contable
Responsable Nacional de Mantenimiento y Soporte de Sistemas
Asesor(a) Legal - Contratos y Licitaciones - Santa Cruz
Auxiliar de Servicios Generales - Cochabamba
Gestor(a) de Compensaciones y Beneficios
Especialista de Sutura - Santa Cruz
Coordinador(a) de Distribución - Cochabamba
Administrador(a)
Diseñador(a) Gráfico(a)
Product Manager BTL / Eventos
Encargado(a) de Almacenes - Santa Cruz
Asistente de Marketing - Santa Cruz
Business Intelligence Analyst
Consignatario(a) y/o Administrador(a) de Tienda - La Paz
Consignatario(a) y/o Administrador(a) de Tienda - Cochabamba
Contador(a) Júnior
Jefe Regional de Operaciones - Santa Cruz
Técnico en Selección de Personal
E

Abogado(a) Regional – Sucre
Asesor(a) de Créditos – El Alto
Asesor(a) de Créditos - La Paz
Encargado(a) Operativo(a) Administrativo(a) – Luribay
Asistente de RR.HH.
Recepcionista - Uyuni
Counter de Reservas - La Paz
Auxiliar de Tesorería
Prácticas Pre Profesionales en Archivo - La Paz
Mensajero(a) / Repartidor(a)
Asesores(as) de Ventas
Contador(a) Júnior
Vendedores(as) - La Paz
Vendedores(as) - Santa Cruz
Ejecutivo(a) de Ventas
Sénior para el Área de Playroll - Santa Cruz
Ejecutivo(a) de Ventas - Santa Cruz de la Sierra
Asesor(a) de Inversiones (Créditos)
Asesor(a) Comercial Inmobiliario(a) - Santa Cruz de la Sierra
Jefe Comercial - Santa Cruz de la Sierra
Veterinario(a) / Encargado(a) de Linea de Producto Veterinario
Recepcionista - Santa Cruz de la Sierra
Practicantes para el Área de Contabilidad y Administración
Ejecutivo(a) Comercial - División Equipos Médicos
Asistente de Recursos Humanos
Gerente General
Analista de Procesos - Tarija
Ejecutivo(a) de Ventas Canal Institucional (HOR

Auxiliar de Pañol - Cochabamba
Auxiliar Comercial
Analista de Control de Gestión
Desarrollador(a) Java
Responsable de Caja y Cobranza
Asesor(a) Comercial - Santa Cruz
Asesores(as) Comerciales
Jefe de Mercadotecnia - Cochabamba
Practicantes Comerciales - Cochabamba
Asesor(a) de Ventas
Practicantes Industriales - Cochabamba
Sub Gerente Comercial
Asesor(a) Comercial de Interiorismo
Agentes de Seguro - Sucre
Agentes de Seguro - Potosí
Cajero(a) Agencia Charagua - Santa Cruz
Oficial de Tarjetas de Crédito - La Paz
Analista de Créditos Especiales (Modificado)
Analista Legal - La Paz
Analista de Organización y Métodos - Santa Cruz
Asistente de Marketing - Santa Cruz
Encargado(a) de Recursos Humanos
Asistente de Tienda
Recepcionista
Servicio Financiero de Sepelio
Asistente de Operaciones - Comercio Exterior - Santa Cruz
Recepcionista - La Paz
Ejecutivo(a) de Ventas
Asistente de Banquetes
Ejecutivo(a) de Cobranzas
Decoradora(or) de Interiores (Modificado)
Supervisor(a) de Ventas - Cochabamba
Of

In [14]:
len(Results)

880

In [15]:
import csv
# Open the file in the mode append and the encoding utf-8
outfile = open("/Users/Roberto/Documents/Python Scripts/Machine Learning Projects/Labor Market/TRABAJOPOLIS_FEB2019.csv", 'a',newline='', encoding = 'utf-8')
writer = csv.writer(outfile)
try:
    for i in range(len(Results)):
        csvRow = []
        # Transform the list shape:
        if Results[i][0] is not None:
            
            for cell in Results[i][0]:
                csvRow.append(cell)
                
            csvRow.append(Results[i][1])
            csvRow.append(Results[i][2])

            writer.writerow(csvRow)
finally:
    outfile.close() 

In [16]:
Results[400]

(['Grupo Empresarial KD',
  '794413',
  'Santa Cruz de la Sierra, Bolivia',
  'Administración y Oficina,Dirección y Gerencia ',
  'De Bs. 5.000 a Bs. 5.999 ',
  '10 Septiembre 2019',
  '10 Octubre 2019',
  'encargado de almacen requisitos conocimiento de manejo de kardex garantia real de inmueble se firmara un documento privado notariado garantizando con su inmueble la mercaderia que estara en su poder en caso de cualquier irregularidad tolerancia de trabajo bajo presion capacidad de trabajar en equipo iniciativa criterio dinamismo eficiencia honestidad y creatividad  funciones recepcion y envio de mercaderia repuestos para camiones y maquinaria organizacion y revision de inventarios del almacen aprender toda la gama de repuestos diesel que ofrecemos bonificacion si existe conocimiento mecanicolas personas interesadas que cumplan con el perfil podran remitir su curriculum vitae indicando su pretension salarial por trabajopolisbo'],
 'Encargado(a) de Almacén',
 'https://www.trabajopolis

In [17]:
Results[0]

(['HANY FLEX S.R.L.',
  '798429',
  'La Paz, Bolivia',
  'Marketing y Ventas ',
  'De Bs. 2.122 a Bs. 2.999 ',
  '25 Septiembre 2019',
  '25 Octubre 2019',
  'vendedor objetivopromocionar y distribuir insumos graficos de la empresa requisitos conocimiento en ventas atencion y captacion de clientes disponibilidad de tiempo capacidad de trabajo en equipose ofrecesueldo fijo los interesados enviar curriculum vitae con pretension salarial por trabajopolisbo '],
 'Vendedor(a)',
 'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/798429/Vendedor(a).html?searchId=1546400657&page=1')

In [18]:
Results = trabajopolis(['cajero'])

The request response cycle was successful
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/797252/Cajero(a)---Cochabamba.html?searchId=1569443464.15&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/794704/Cajero(a)---Agencias-Urbanas.html?searchId=1569443464.15&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/794792/Cajero(a)-Procesador(a)---Potosí.html?searchId=1569443464.15&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/790913/Cajero(a).html?searchId=1569443464.15&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/792102/Cajero(a)---Santa-Cruz.html?searchId=1569443464.15&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/792082/Cajero(a)---Santa-Cruz.html?searchId=1569443464.15&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/790877/Cajero(a)-Agencia-Charagua---Santa-Cruz.html?searchId=1569443464.15&page=1
https://www.trabajopo

NameError: name 'time' is not defined

In [ ]:
hola = Results[1000:]

In [ ]:
for i in range(len(hola)-1):
    csvRow = []
        
    if Results[i][0] is not None:
            
        for cell in hola[i][0]:
            csvRow.append(cell)
                
        csvRow.append(hola[i][1])
        csvRow.append(hola[i][2])

print(csvRow)

In [53]:
csvrow = []
for cell in Results[i][0]:
    #print(cell)
    csvrow.append(cell)
csvrow.append(Results[i][1])
csvrow.append(Results[i][2])
print(csvrow)

['Guayabochi srl', '699870', 'Santa Cruz de la Sierra, Bolivia', 'Administración y Oficina ', 'No Declarado ', '07 Enero 2019', '06 Febrero 2019', 'empresa de servicios busca incorporar a su equipo de trabajo el cargomensajerorequisitos bachiller yo estudiante de carreras administrativas buen manejo de microsoft office buena ortografia y redaccion disponibilidad de viajes trabajo bajo presion disponibilidad inmediataofrecemos buen ambiente de trabajo remuneracion de acorde al cargolas personas que cumplan con el perfil mencionado postularse enviando su pretension salarial hoja de vida y fotografia actualizada por trabajopolisbo', 'Mensajero(a)', 'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/699870/Mensajero(a).html?searchId=1546400657&page=1']


In [43]:
Results[i][0]

['Guayabochi srl',
 '699870',
 'Santa Cruz de la Sierra, Bolivia',
 'Administración y Oficina ',
 'No Declarado ',
 '07 Enero 2019',
 '06 Febrero 2019',
 'empresa de servicios busca incorporar a su equipo de trabajo el cargomensajerorequisitos bachiller yo estudiante de carreras administrativas buen manejo de microsoft office buena ortografia y redaccion disponibilidad de viajes trabajo bajo presion disponibilidad inmediataofrecemos buen ambiente de trabajo remuneracion de acorde al cargolas personas que cumplan con el perfil mencionado postularse enviando su pretension salarial hoja de vida y fotografia actualizada por trabajopolisbo']

In [44]:
for cell in Results[i][0]:
    print(cell)

Guayabochi srl
699870
Santa Cruz de la Sierra, Bolivia
Administración y Oficina 
No Declarado 
07 Enero 2019
06 Febrero 2019
empresa de servicios busca incorporar a su equipo de trabajo el cargomensajerorequisitos bachiller yo estudiante de carreras administrativas buen manejo de microsoft office buena ortografia y redaccion disponibilidad de viajes trabajo bajo presion disponibilidad inmediataofrecemos buen ambiente de trabajo remuneracion de acorde al cargolas personas que cumplan con el perfil mencionado postularse enviando su pretension salarial hoja de vida y fotografia actualizada por trabajopolisbo


In [45]:
csvRow.append(cell)